### Export annotations from Labelbox to GCP storage

In [2]:
import os, sys
sys.path.insert(0, os.path.abspath('..'))

%load_ext autoreload
%autoreload 2

In [3]:
%reload_ext autoreload

In [4]:
from google.cloud import storage
from labelbox import Client as LabelboxClient

from utils import gcp_utils
from utils.data_management import get_annotation_path
from utils.labelbox_utils import get_annotation_objects_from_data_row_export, get_geojson_fc_from_annotation_objects, get_confidence_geojson_fc_from_annotation_objects

2023-10-17 16:30:15:rastervision.pipeline.rv_config: WARNING - Root temporary directory cannot be used: /data/tmp. Using root: /data/tmp/tmpulp9xr_c


In [5]:
# Load environment variables from .env.
# Alternatively, manually set environment variables.

from dotenv import load_dotenv
load_dotenv()

True

In [37]:

from project_config import LABELBOX_PROJECT_ID, GCP_PROJECT_NAME
LABELBOX_API_KEY = os.getenv('LABELBOX_API_KEY')

gcp_client = storage.Client(project=GCP_PROJECT_NAME)
labelbox_client = LabelboxClient(api_key=LABELBOX_API_KEY)
project = labelbox_client.get_project(LABELBOX_PROJECT_ID)


In [7]:
# CAREFUL
# Use with caution

# The following deletes all files in our GCP bucket with a _annotations.geojson file ending

"""
from project_config import BUCKET_NAME

bucket = gcp_client.get_bucket(BUCKET_NAME)
for blob in bucket.list_blobs():
    if blob.name.endswith("_annotations.geojson"):
        print(f"Deleting {blob} .. ")
        blob.delete()
"""

'\nfrom project_config import BUCKET_NAME\n\nbucket = gcp_client.get_bucket(BUCKET_NAME)\nfor blob in bucket.list_blobs():\n    if blob.name.endswith("_annotations.geojson"):\n        print(f"Deleting {blob} .. ")\n        blob.delete()\n'

In [38]:
export_params = {"project_details": True}
export = project.export_v2(params=export_params)

export.wait_till_done()
if export.errors:
  print("Errors:")
  print(export.errors)
else:
  export_full_project = export.result
  print(f"Finished without errors. Total export contains {len(export_full_project)} data rows.")
  print(len(export_full_project))

Finished without errors. Total export contains 186 data rows.
186


In [8]:
# from project_config import LABELBOX_PROJECT_ID

# for data_row_export in export_full_project:
#     global_key_data_row = data_row_export['data_row']['global_key']

#     workflow_status = data_row_export['projects'][LABELBOX_PROJECT_ID]['project_details']['workflow_status']
#     # We filter depending on workflow_status
#     if workflow_status in ["IN_REWORK", "TO_LABEL", "IN_REVIEW"]:
#         continue
#     assert workflow_status == "DONE"
    
#     print(f"Processing annotations for {global_key_data_row}")
#     annotated_objects = get_annotation_objects_from_data_row_export(data_row_export)
    
#     geojson_feature_collection = get_geojson_fc_from_annotation_objects(annotated_objects)
#     annotation_destination_path = get_annotation_path(global_key_data_row)
#     gcp_utils.upload_json(gcp_client, geojson_feature_collection, annotation_destination_path)
    

Processing annotations for Kathajodi_Cuttack_85-85_20-44_2022-05-01_rgb_median, cllbefzr706lu075o4tjp2vil
Processing annotations for Kathajodi_Cuttack_85-85_20-44_2022-02-01_rgb_median, cllbefzr706ly075o3nf0aus5
Processing annotations for Ken_Banda_80-35_25-68_2022-06-01_rgb_median, cllbeg46n065b072lhlezgv5e
Processing annotations for Ken_Banda_80-35_25-68_2022-01-01_rgb_median, cllbeg46n065n072l0f976hg4
Processing annotations for Narmada_Sehore_77-32_22-56_2022-01-01_rgb_median, cllbegm6p060v073560lcg1p9
Processing annotations for Narmada_Sehore_77-32_22-56_2022-10-01_rgb_median, cllbegm6p060z0735b5n8e34p
Processing annotations for Tawa_Hoshangabad_77-80_22-74_2022-04-01_rgb_median, cllbegpkw01yf076j7yby2fmn
Processing annotations for Tawa_Hoshangabad_77-80_22-74_2022-01-01_rgb_median, cllbegpkw01yv076j41v97lgj
Processing annotations for Sone_Rohtas_84-21_24-91_2022-02-01_rgb_median, cllbegsyt01bk0786d79nbkp0
Processing annotations for Sone_Rohtas_84-21_24-91_2022-05-01_rgb_median, cl

In [40]:
from project_config import LABELBOX_PROJECT_ID

annotated_objects= None

for data_row_export in export_full_project:
    global_key_data_row = data_row_export['data_row']['global_key']

    workflow_status = data_row_export['projects'][LABELBOX_PROJECT_ID]['project_details']['workflow_status']
    # We filter depending on workflow_status
    # if workflow_status in ["IN_REWORK", "TO_LABEL", "IN_REVIEW"]:
    #     continue
    # assert workflow_status == "DONE"

    if data_row_export['data_row']['id'] == 'clmjq31xe0qpy073u29vcbtd3':
        print ("Found It")
        
    
        print(f"Processing annotations for {global_key_data_row}")
        annotated_objects = get_annotation_objects_from_data_row_export(data_row_export)
        
        # geojson_feature_collection = get_geojson_fc_from_annotation_objects(annotated_objects)
        geojson_feature_collection = get_confidence_geojson_fc_from_annotation_objects(annotated_objects)
        annotation_destination_path = get_annotation_path(global_key_data_row)
        gcp_utils.upload_json(gcp_client, geojson_feature_collection, annotation_destination_path)

        #print annotated_objects in json format
        # import json
        # print(json.dumps(geojson_feature_collection, indent=4))



        #exit loop
        break

    

Found It
Processing annotations for Godavari_EastGodavari_81-84_16-66_2022-06-01_rgb_median


In [29]:
print(json.dumps(annotated_objects, indent=4))

[
    {
        "feature_id": "clntiawxw00012e6got8n4pj5",
        "feature_schema_id": "clnclx10u0ray073aef2ycqdz",
        "name": "sandmine-confidence",
        "annotation_kind": "GeoPolygon",
        "classifications": [
            {
                "feature_id": "clntibofh00042e6gu4wfv5mm",
                "feature_schema_id": "clnclx10u0raz073a2pexflvk",
                "name": "Confidence Level",
                "radio_answer": {
                    "feature_id": "clntibofh00032e6gszen3k7y",
                    "feature_schema_id": "clnclx10u0rb0073a018q69ca",
                    "name": "Very High (100%)",
                    "classifications": []
                }
            }
        ],
        "geojson": {
            "type": "Polygon",
            "coordinates": [
                [
                    [
                        81.86003208160402,
                        16.600106571682208
                    ],
                    [
                        81.861534118652

In [30]:
polygons = [o['geojson'] for o in annotated_objects]
polygons

[{'type': 'Polygon',
  'coordinates': [[[81.86003208160402, 16.600106571682208],
    [81.86153411865236, 16.60094966760054],
    [81.86340093612672, 16.60004488159175],
    [81.86374425888063, 16.598975583544398],
    [81.86344385147096, 16.597865152352682],
    [81.86322927474977, 16.595479759954276],
    [81.86297178268433, 16.593649568134367],
    [81.86239242553712, 16.592621362839772],
    [81.86144828796388, 16.593258850770127],
    [81.86084747314455, 16.593814080471244],
    [81.86043977737428, 16.59506848241128],
    [81.86005353927614, 16.59607611082881],
    [81.85994625091554, 16.596939788125997],
    [81.86003208160402, 16.600106571682208]]],
  'properties': {'confidence': 'Very High (100%)'}},
 {'type': 'Polygon',
  'coordinates': [[[81.85827255249025, 16.594595512149162],
    [81.860568523407, 16.594842279387617],
    [81.86074018478395, 16.59451325633264],
    [81.85983896255493, 16.59426648867188],
    [81.85971021652223, 16.59346449158711],
    [81.85940980911256, 16.

{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [
                    -80.870885,
                    35.215151
                ]
            },
            "properties": {
                "name": "ABBOTT NEIGHBORHOOD PARK",
                "address": "1300  SPRUCE ST"
            }
        }
    ]
}